In [3]:
import os
import numpy as np

import laspy
#import open3d as o3d

import rasterio
from rasterio.transform import from_origin
from scipy.stats import entropy

#### Data Imports

In [ ]:
las_file = "C:/Users/harre/RS_Technical/ALS_pointcloud.las" #Replace with local filepath
downsampled_dir = "C:/Users/harre/RS_Technical/"
downsampled_las = "../structural_metrics/data/downsampled.las"
output_dir = "../structural_metrics/data"
grid_resolution = 1.0
crs_epsg = "EPSG:32637"  # <-- update with your CRS

In [5]:
os.path.realpath("../structural_metrics/data/downsampled.las")

'C:\\Users\\harre\\RS_Analyst_Technical_Assignments\\structural_metrics\\data\\downsampled.las'

In [6]:
def voxel_downsample_las(las_file: str, output_dir: str, voxel_size: float = 1.0):
    """
    Downsamples a LAS/LAZ file using voxel grid filtering and saves the result.
    
    Parameters:
        las_file (str): Path to the input .las or .laz file.
        output_dir (str): Directory to save the downsampled .las file.
        voxel_size (float): Voxel size in meters. Default is 1.0.
        
    Output:
        Saves a downsampled LAS file as 'downsampled.las' in output_dir.
    """
    os.makedirs(output_dir, exist_ok=True)

    print(f"Reading: {las_file}")
    las = laspy.read(las_file)
    points = np.vstack((las.x, las.y, las.z)).T
    print(f"Original point count: {points.shape[0]}")

    print(f"Downsampling with voxel size: {voxel_size} m")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
    down_points = np.asarray(pcd_down.points)
    print(f"Downsampled point count: {down_points.shape[0]}")

    # Create new LAS object and copy header info
    down_las = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
    
    # Set coordinates
    down_las.x = down_points[:, 0]
    down_las.y = down_points[:, 1]
    down_las.z = down_points[:, 2]

    # Save file
    out_path = os.path.join(output_dir, "downsampled.las")
    down_las.write(out_path)
    print(f"Downsampled LAS file saved to: {out_path}")

In [ ]:
#downsampled = voxel_downsample_las(las_file=las_file, output_dir=downsampled_dir)

In [9]:
# Read LAS file
las = laspy.read(downsampled_las)

def fetch_las_metadata(las_file: str):
    """ Print basic metadata """
    print(f"File: {las_file}")
    print(f"Point format: {las.header.point_format}")
    print(f"Point count: {las.header.point_count}")
    print(f"Bounds (XYZ):")
    print(f"  X: {las.header.min[0]} to {las.header.max[0]}")
    print(f"  Y: {las.header.min[1]} to {las.header.max[1]}")
    print(f"  Z: {las.header.min[2]} to {las.header.max[2]}")

    # Print available attributes (e.g., return number, intensity)
    print("\nAvailable dimensions:")
    print(list(las.point_format.dimension_names))


    try:
        crs = las.header.parse_crs()
        if crs:
            print("\nCoordinate Reference System (CRS):")
            print(crs)
        else:
            print("\nCRS not embedded in the LAS file.")
    except AttributeError:
        print("\nCRS extraction not supported for this LAS version or library version.")

fetch_las_metadata(las)

File: <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 2976899 points, 0 vlrs)>
Point format: <PointFormat(3, 0 bytes of extra dims)>
Point count: 2976899
Bounds (XYZ):
  X: 284777.74 to 285291.19
  Y: 18664.83 to 20903.78
  Z: 1797.07 to 1925.89

Available dimensions:
['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'gps_time', 'red', 'green', 'blue']

CRS not embedded in the LAS file.


In [10]:
las = laspy.read(downsampled_las)

x = las.x
y = las.y
z = las.z
return_nums = las.return_number
num_returns = las.number_of_returns

In [11]:
xmin, ymin = np.min(x), np.min(y)
xmax, ymax = np.max(x), np.max(y)

ncols = int(np.ceil((xmax - xmin) / grid_resolution))
nrows = int(np.ceil((ymax - ymin) / grid_resolution))

def to_grid_coords(x, y):
    col = ((x - xmin) / grid_resolution).astype(int)
    row = ((ymax - y) / grid_resolution).astype(int)  # flip y-axis
    return row, col

rows, cols = to_grid_coords(x, y)

In [12]:
def init_grid():
    return np.full((nrows, ncols), np.nan)

chm = init_grid()
height_sd = init_grid()
p25 = init_grid()
p50 = init_grid()
p75 = init_grid()
p95 = init_grid()
mean_height = init_grid()
canopy_cover = init_grid()
vci = init_grid()
height_entropy = init_grid()
point_density = np.zeros((nrows, ncols))  # no NaN, count-based

In [ ]:
for r in range(nrows):
    for c in range(ncols):
        mask = (rows == r) & (cols == c)
        if not np.any(mask):
            continue

        cell_z = z[mask]
        if len(cell_z) == 0:
            continue

        cell_returns = return_nums[mask]
        point_density[r, c] = len(cell_z)

        chm[r, c] = np.max(cell_z)
        height_sd[r, c] = np.std(cell_z)

        # Canopy analysis (returns > 2m)
        threshold = 2.0
        above = cell_z[cell_z > threshold]
        if len(above) > 0:
            canopy_cover[r, c] = len(above) / len(cell_z)
            mean_height[r, c] = np.mean(above)
            p25[r, c] = np.percentile(above, 25)
            p50[r, c] = np.percentile(above, 50)
            p75[r, c] = np.percentile(above, 75)
            p95[r, c] = np.percentile(above, 95)
            mid = cell_z[(cell_z > 2) & (cell_z < 10)]
            vci[r, c] = len(mid) / len(above) if len(above) > 0 else np.nan
        else:
            canopy_cover[r, c] = 0

        # Entropy of height distribution
        hist = np.histogram(cell_z, bins=5, range=(0, np.max(cell_z)))[0]
        if hist.sum() > 0:
            height_entropy[r, c] = entropy(hist)

In [ ]:
transform = from_origin(xmin, ymax, grid_resolution, grid_resolution)

def save_raster(array, name):
    path = os.path.join(output_dir, f"{name}.tif")
    with rasterio.open(
        path,
        "w",
        driver="GTiff",
        height=array.shape[0],
        width=array.shape[1],
        count=1,
        dtype="float32",
        crs=crs_epsg,
        transform=transform,
        nodata=np.nan
    ) as dst:
        dst.write(array.astype("float32"), 1)
    print(f"Saved: {path}")

In [ ]:
# Save all metric rasters
save_raster(chm, "chm")
save_raster(height_sd, "height_stddev")
save_raster(p25, "height_p25")
save_raster(p50, "height_p50")
save_raster(p75, "height_p75")
save_raster(p95, "height_p95")
save_raster(mean_height, "mean_canopy_height")
save_raster(canopy_cover, "canopy_cover")
save_raster(vci, "vertical_complexity_index")
save_raster(height_entropy, "height_entropy")
save_raster(point_density, "point_density")